In [ ]:
#pip install openai httpx python-dotenv langgraph langchain_openai langchain_community ipython

In [11]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [12]:
from dotenv import load_dotenv
_ = load_dotenv()

In [13]:
model = "gpt-3.5-turbo"

In [14]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [15]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [16]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [17]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model=model)  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [ ]:
#untuk display struktur graph
# tapi error gak bisa install package pygraphviz yang dibutuhin "pip install pygraphviz"
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

In [26]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_WpEBUDC7DUb4kduBOncnZV0W'}
Back to the model!


In [27]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WpEBUDC7DUb4kduBOncnZV0W', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e0a730d4-267f-4a41-b8bb-c445f1bd8ec0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_WpEBUDC7DUb4kduBOncnZV0W'}], usage_metadata={'input_tokens': 153, 'output_tokens': 21, 'total_tokens': 174}),
  ToolMessage(content='[{\'url\': \'https://www.msn.com/en-us/weather/topstories/june-26-2024-san-francisco-bay-area-weather-forecast/vi-BB1oVEx8\', \'content\': \'June 26, 2024 San Francisco Bay Area weather forecast. Po

In [28]:
result['messages'][-1].content

'The current weather in San Francisco is overcast with a temperature of 55.9°F (13.3°C). The wind speed is 16.9 kph coming from the west, and the humidity is at 93%.'

In [29]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_4t5wAyT1RFriGbHHf4e496Os'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_gkK54JPC0Coy84Be02FdfYzu'}
Back to the model!


In [30]:
result['messages'][-1].content

'The current weather in San Francisco is overcast with a temperature of 55.9°F. The wind speed is 16.9 kph coming from the west. The humidity is at 93%.\n\nIn Los Angeles, the current weather is sunny with a temperature of 66.0°F. The wind speed is 3.6 kph coming from the north. The humidity is at 81%.'

In [32]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl winner 2024'}, 'id': 'call_ShEgrTl7DjQbAtacOzaN0u6g'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'State of Super Bowl 2024 winning team headquarters'}, 'id': 'call_XlPYzZsvYqpVliAkrCkpRyUS'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of state of Super Bowl 2024 winning team headquarters'}, 'id': 'call_41U7B3JnwWUL8FL5NzsgClLW'}
Back to the model!


In [33]:
result['messages'][-1].content

"1. **Who won the Super Bowl in 2024?**\n   - The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers 25-22 in overtime.\n\n2. **In what state is the winning team's headquarters located?**\n   - The Kansas City Chiefs' headquarters is located in the state of Missouri.\n\n3. **What is the GDP of that state?**\n   - The GDP of Missouri, the state where the Kansas City Chiefs are headquartered, was approximately $364.5 billion in 2022. It's advisable to check for the most current figures as they can change annually."